**Transfer Learning**

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
train_data_dir = 'C:/Users/subhayu/Desktop/Few Shot Learning/fewShot(ShoeScandalBoot)/Train'
test_data_dir = 'C:/Users/subhayu/Desktop/Few Shot Learning/fewShot(ShoeScandalBoot)/Test'

img_width, img_height = 128,128

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=32,
                                                  class_mode='categorical')

num_classes = 3

train_labels = to_categorical(train_generator.classes, num_classes)
test_labels = to_categorical(test_generator.classes, num_classes)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
curr = time.time()
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=50,
          validation_data=test_generator,
          validation_steps=len(test_generator))
stop=time.time()
print("time is",stop-curr)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = test_generator.classes

report = classification_report(y_true_classes, y_pred_classes, target_names=test_generator.class_indices)

accuracy = np.mean(y_pred_classes == y_true_classes)
print(f"Accuracy: {accuracy*100:.2f}%")

print("Classification Report:")
print(report)

**Meta Learning (Few Shot Learning Code)**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import os
import matplotlib.pyplot as plt
import numpy as np
import time

train_data_dir = 'C:/Users/subhayu/Desktop/Few Shot Learning/fewShot(Chest Xray)/Train'
test_data_dir = 'C:/Users/subhayu/Desktop/Few Shot Learning/fewShot(Chest Xray)/Test'

In [ ]:
num_classes=2
learning_rate = 0.003
meta_step_size = 0.25

inner_batch_size = 25
eval_batch_size = 25

meta_iters = 2000
eval_iters = 5
inner_iters = 4

eval_interval = 1
train_shots = 20
shots = 5

classes = num_classes
img_height, img_width = 28, 28

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
def conv_bn(x):
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    return layers.ReLU()(x)

def build_prototypical_net(input_shape, num_classes):
    inputs = layers.Input(input_shape)
    x = conv_bn(inputs)
    x = conv_bn(x)
    x = conv_bn(x)
    x = conv_bn(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inputs, outputs)

model = build_prototypical_net((img_height, img_width, 3), num_classes)

learning_rate = 0.001
optimizer = optimizers.SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
training = []
testing = []

curr = time.time()
for meta_iter in range(meta_iters):
    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    old_vars = model.get_weights()
    mini_dataset = iter(train_generator)
    for i in range(inner_iters):
        images, labels = next(mini_dataset)
        with tf.GradientTape() as tape:
            preds = model(images)
            loss = tf.keras.losses.categorical_crossentropy(labels, preds)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    new_vars = model.get_weights()
    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + ((new_vars[var] - old_vars[var]) * cur_meta_step_size)
    model.set_weights(new_vars)
    if meta_iter % eval_interval == 0:
        accuracies = []
        for dataset in (train_data_dir, test_data_dir):
            data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
            dataset = data_gen.flow_from_directory(
                dataset,
                target_size=(img_height, img_width),
                batch_size=eval_batch_size,
                class_mode='categorical',
                shuffle=False
            )
            images, labels = next(dataset)
            old_vars = model.get_weights()
            for i in range(inner_iters):
                with tf.GradientTape() as tape:
                    preds = model(images)
                    loss = tf.keras.losses.categorical_crossentropy(labels, preds)
                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
            test_preds = model.predict(images)
            test_preds = tf.argmax(test_preds, axis=1).numpy()
            num_correct = (test_preds == tf.argmax(labels, axis=1).numpy()).sum()
            model.set_weights(old_vars)
            accuracies.append(num_correct / eval_batch_size)
        training.append(accuracies[0])
        testing.append(accuracies[1])
        if meta_iter % 100 == 0:
            print("batch %d: train=%f test=%f" % (meta_iter, accuracies[0], accuracies[1]))

stop=time.time()
test_accuracy = testing[-1]
print(f'Final Test accuracy: {test_accuracy * 100:.2f}%')
print(f'Final Time : {stop-curr}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_data_gen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

true_labels = test_generator.classes
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

f1=2 * ((precision * recall) / (precision + recall))

print(f'Final Precision: {precision}')
print(f'Final Recall: {recall}')
print(f'Final F1-score: {f1}')

In [ ]:
window_length = 100
train_s3 = np.r_[training[window_length - 1:0:-1], training, training[-1:-window_length:-1]]
test_s3 = np.r_[testing[window_length - 1:0:-1], testing, testing[-1:-window_length:-1]]
w = np.hamming(window_length)
train_y3 = np.convolve(w / w.sum(), train_s3, mode="valid")
test_y3 = np.convolve(w / w.sum(), test_s3, mode="valid")
x3 = np.arange(0, len(test_y3), 1)
plt.plot(x3,train_y3,x3,test_y3)
plt.legend(["test", "train"])
plt.grid()
plt.xlabel("Meta-iterations")
plt.ylabel("Accuracy")
plt.title("Moving Average Accuracy")
plt.show()

**Basic CNN Model**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import classification_report

In [ ]:
train_data_dir = 'C:/Users/subhayu/Desktop/Few Shot Learning/fewShot(ShoeScandalBoot)/Train'
test_data_dir = 'C:/Users/subhayu/Desktop/Few Shot Learning/fewShot(ShoeScandalBoot)/Test'

img_width, img_height = 128,128

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=32,
                                                  class_mode='categorical')

num_classes = 3

train_labels = to_categorical(train_generator.classes, num_classes)
test_labels = to_categorical(test_generator.classes, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
curr = time.time()
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=50,
          validation_data=test_generator,
          validation_steps=len(test_generator))
stop=time.time()
print("time is",stop-curr)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true_classes = test_generator.classes

report = classification_report(y_true_classes, y_pred_classes, target_names=test_generator.class_indices)

accuracy = np.mean(y_pred_classes == y_true_classes)
print(f"Accuracy: {accuracy*100:.2f}%")

print("Classification Report:")
print(report)